In [2]:
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [3]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH11-Retriever")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH11-Retriever


# 샘플 데이터 생성

In [4]:
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

# 화장품 상품의 설명과 메타데이터 생성
docs = [
    Document(
        page_content="수분 가득한 히알루론산 세럼으로 피부 속 깊은 곳까지 수분을 공급합니다.",
        metadata={"year": 2024, "category": "스킨케어", "user_rating": 4.7},
    ),
    Document(
        page_content="24시간 지속되는 매트한 피니시의 파운데이션, 모공을 커버하고 자연스러운 피부 표현이 가능합니다.",
        metadata={"year": 2023, "category": "메이크업", "user_rating": 4.5},
    ),
    Document(
        page_content="식물성 성분으로 만든 저자극 클렌징 오일, 메이크업과 노폐물을 부드럽게 제거합니다.",
        metadata={"year": 2023, "category": "클렌징", "user_rating": 4.8},
    ),
    Document(
        page_content="비타민 C 함유 브라이트닝 크림, 칙칙한 피부톤을 환하게 밝혀줍니다.",
        metadata={"year": 2023, "category": "스킨케어", "user_rating": 4.6},
    ),
    Document(
        page_content="롱래스팅 립스틱, 선명한 발색과 촉촉한 사용감으로 하루종일 편안하게 사용 가능합니다.",
        metadata={"year": 2024, "category": "메이크업", "user_rating": 4.4},
    ),
    Document(
        page_content="자외선 차단 기능이 있는 톤업 선크림, SPF50+/PA++++ 높은 자외선 차단 지수로 피부를 보호합니다.",
        metadata={"year": 2024, "category": "선케어", "user_rating": 4.9},
    ),
]

# 벡터 저장소 생성
vectorstore = Chroma.from_documents(
    docs, OpenAIEmbeddings(model="text-embedding-3-small")
)

# SelfQueryRetriever

In [5]:
from langchain.chains.query_constructor.base import AttributeInfo

# 메타데이터 필드 정보 생성
metadata_field_info = [
    AttributeInfo(
        name="category",
        description="The category of the cosmetic product. One of ['스킨케어', '메이크업', '클렌징', '선케어']",
        type="string"
    ),
    AttributeInfo(
        name="year",
        description="The year the cosmetic product was released",
        type="integer",
    ),
    AttributeInfo(
        name="user_rating",
        description="A user rating for the cosmetic product, ranging form 1 to 5",
        type="float",
    )
]

In [6]:
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import ChatOpenAI

# LLM 정의
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# SelfQueryRetriever 생성
retriever = SelfQueryRetriever.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    document_contents="Brief summary of a cosmetic product",
    metadata_field_info=metadata_field_info
)

# Query 테스트

In [7]:
# Self-query 검색
retriever.invoke("평점이 4.8 이상인 제품을 추천해주세요.")

[Document(id='5b5b41a1-2c1f-4445-bdae-45f1bdf56d47', metadata={'category': '선케어', 'user_rating': 4.9, 'year': 2024}, page_content='자외선 차단 기능이 있는 톤업 선크림, SPF50+/PA++++ 높은 자외선 차단 지수로 피부를 보호합니다.'),
 Document(id='787f6bd0-6868-493e-843c-cfd8b63371bf', metadata={'category': '클렌징', 'user_rating': 4.8, 'year': 2023}, page_content='식물성 성분으로 만든 저자극 클렌징 오일, 메이크업과 노폐물을 부드럽게 제거합니다.')]

In [8]:
# Self-query 검색
retriever.invoke("2023년에 출시된 상품을 추천해주세요.")

[Document(id='4cb71868-e8eb-44ef-b280-76e37a4ed9ca', metadata={'category': '메이크업', 'user_rating': 4.5, 'year': 2023}, page_content='24시간 지속되는 매트한 피니시의 파운데이션, 모공을 커버하고 자연스러운 피부 표현이 가능합니다.'),
 Document(id='f2b705c2-bac8-458e-b4b9-f0aef07dbaeb', metadata={'category': '스킨케어', 'user_rating': 4.6, 'year': 2023}, page_content='비타민 C 함유 브라이트닝 크림, 칙칙한 피부톤을 환하게 밝혀줍니다.'),
 Document(id='787f6bd0-6868-493e-843c-cfd8b63371bf', metadata={'category': '클렌징', 'user_rating': 4.8, 'year': 2023}, page_content='식물성 성분으로 만든 저자극 클렌징 오일, 메이크업과 노폐물을 부드럽게 제거합니다.')]

In [9]:
# Self-query 검색
retriever.invoke("카테고리가 선케어인 상춤을 추천해주세요")

[Document(id='5b5b41a1-2c1f-4445-bdae-45f1bdf56d47', metadata={'category': '선케어', 'user_rating': 4.9, 'year': 2024}, page_content='자외선 차단 기능이 있는 톤업 선크림, SPF50+/PA++++ 높은 자외선 차단 지수로 피부를 보호합니다.'),
 Document(id='fba5f56f-8f31-4dd2-835d-36e323eb8544', metadata={'category': '메이크업', 'user_rating': 4.4, 'year': 2024}, page_content='롱래스팅 립스틱, 선명한 발색과 촉촉한 사용감으로 하루종일 편안하게 사용 가능합니다.'),
 Document(id='4cb71868-e8eb-44ef-b280-76e37a4ed9ca', metadata={'category': '메이크업', 'user_rating': 4.5, 'year': 2023}, page_content='24시간 지속되는 매트한 피니시의 파운데이션, 모공을 커버하고 자연스러운 피부 표현이 가능합니다.'),
 Document(id='787f6bd0-6868-493e-843c-cfd8b63371bf', metadata={'category': '클렌징', 'user_rating': 4.8, 'year': 2023}, page_content='식물성 성분으로 만든 저자극 클렌징 오일, 메이크업과 노폐물을 부드럽게 제거합니다.')]

In [10]:
# Self-query 검색
retriever.invoke(
    "카테고리가 메이크업인 상품 중에서 평점이 4.5 이상인 상품을 추천해주세요"
)

[Document(id='4cb71868-e8eb-44ef-b280-76e37a4ed9ca', metadata={'category': '메이크업', 'user_rating': 4.5, 'year': 2023}, page_content='24시간 지속되는 매트한 피니시의 파운데이션, 모공을 커버하고 자연스러운 피부 표현이 가능합니다.')]

In [11]:
retriever = SelfQueryRetriever.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    document_contents="Brief summary of a cosmetic product",
    metadata_field_info=metadata_field_info,
    enable_limit=True,
    search_kwargs={"k": 2}
)

In [12]:
# Self-query ㄱ머색
retriever.invoke("2023년에 출시된 상품을 추천해주세요.")

[Document(id='4cb71868-e8eb-44ef-b280-76e37a4ed9ca', metadata={'category': '메이크업', 'user_rating': 4.5, 'year': 2023}, page_content='24시간 지속되는 매트한 피니시의 파운데이션, 모공을 커버하고 자연스러운 피부 표현이 가능합니다.'),
 Document(id='f2b705c2-bac8-458e-b4b9-f0aef07dbaeb', metadata={'category': '스킨케어', 'user_rating': 4.6, 'year': 2023}, page_content='비타민 C 함유 브라이트닝 크림, 칙칙한 피부톤을 환하게 밝혀줍니다.')]

In [13]:
retriever = SelfQueryRetriever.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    document_contents="Brief summary of a cosmetic product",
    metadata_field_info=metadata_field_info,
    enable_limit=True,  # 검색 결과 제한 기능을 활성화합니다.
)

# Self-query 검색
retriever.invoke("2023년에 출시된 상품 1개를 추천해주세요")

[Document(id='4cb71868-e8eb-44ef-b280-76e37a4ed9ca', metadata={'category': '메이크업', 'user_rating': 4.5, 'year': 2023}, page_content='24시간 지속되는 매트한 피니시의 파운데이션, 모공을 커버하고 자연스러운 피부 표현이 가능합니다.')]

In [14]:
# Self-query 검색
retriever.invoke("2023년에 출시된 상품 2개를 추천해주세요")

[Document(id='4cb71868-e8eb-44ef-b280-76e37a4ed9ca', metadata={'category': '메이크업', 'user_rating': 4.5, 'year': 2023}, page_content='24시간 지속되는 매트한 피니시의 파운데이션, 모공을 커버하고 자연스러운 피부 표현이 가능합니다.'),
 Document(id='f2b705c2-bac8-458e-b4b9-f0aef07dbaeb', metadata={'category': '스킨케어', 'user_rating': 4.6, 'year': 2023}, page_content='비타민 C 함유 브라이트닝 크림, 칙칙한 피부톤을 환하게 밝혀줍니다.')]

# 더 깊게 들어가기

In [15]:
from langchain.chains.query_constructor.base import StructuredQueryOutputParser, get_query_constructor_prompt

# 문서 내용 설명과 메타데이터 필드 정보를 사용하여 쿼리 생성기 프롬프트를 가져옵니다.
prompt = get_query_constructor_prompt(
    "Brief summary of a cosmetic product", #문서 내용 설명
    metadata_field_info,
)

# StructuredQueryOutputParser 생성
output_parser = StructuredQueryOutputParser.from_components()

# query_constructor chain을 생성
query_constructor = prompt | llm | output_parser

In [16]:
query_output = query_constructor.invoke(
    {
        "query": "2023년도에 출시한 상품 중 평점이 4.5 이상인 상품중에서 스킨케어 제품을 추천해주세요."
    }
)

In [17]:
query_output.filter.arguments

[Comparison(comparator=<Comparator.GTE: 'gte'>, attribute='year', value=2023),
 Comparison(comparator=<Comparator.GTE: 'gte'>, attribute='user_rating', value=4.5),
 Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='category', value='스킨케어')]

In [18]:
from langchain.retrievers.self_query.chroma import ChromaTranslator

retriever = SelfQueryRetriever(
    query_constructor=query_constructor,  # 이전에 생성한 query_constructor chain 을 지정
    vectorstore=vectorstore,  # 벡터 저장소를 지정
    structured_query_translator=ChromaTranslator(),  # 쿼리 변환기
)

In [19]:
retriever.invoke(
    # 질문
    "2023년도에 출시한 상품 중 평점이 4.5 이상인 상품중에서 스킨케어 제품을 추천해주세요"
)

[Document(id='f2b705c2-bac8-458e-b4b9-f0aef07dbaeb', metadata={'category': '스킨케어', 'user_rating': 4.6, 'year': 2023}, page_content='비타민 C 함유 브라이트닝 크림, 칙칙한 피부톤을 환하게 밝혀줍니다.'),
 Document(id='0eba170d-480c-4a20-ab14-dfc06303eb4a', metadata={'category': '스킨케어', 'user_rating': 4.7, 'year': 2024}, page_content='수분 가득한 히알루론산 세럼으로 피부 속 깊은 곳까지 수분을 공급합니다.')]